# LLM-Based D&D Simulation Tutorial

This notebook demonstrates how to use the LLM-based D&D simulation system to create AI-driven gameplay sessions based on real human campaign data.

## Overview

The simulation system includes:
- **Campaign Parameter Extraction**: Analyze human campaigns to extract initialization parameters
- **Character Creation**: Generate D&D characters with personalities and classes
- **Turn Sampling**: Create realistic turn sequences using various sampling methods
- **Character Agents**: LLM-powered characters with memory and decision-making
- **Game Session Management**: Orchestrate complete D&D scenarios

## Setup and Imports

In [1]:
import os
import sys
import json
import random
from pathlib import Path
import textwrap
import numpy as np
import datetime
import os, sys
from collections import Counter

# Import our simulation system
from dnd_dynamics.llm_scaffolding import dnd_simulation as sim
import dnd_dynamics

# Set random seed for reproducible results
#random.seed(42)


Anthropic API key loaded
OpenAI API key loaded
Gemini API key loaded


## Model Selection

The simulation system supports multiple LLM providers through liteLLM. You can use:

- **Anthropic Claude**: `claude-3-5-sonnet-latest`, `claude-3-haiku-latest`
- **OpenAI GPT**: `gpt-4o`, `gpt-4o-mini`, `gpt-3.5-turbo`
- **Google Gemini**: `gemini-pro`, `gemini-pro-vision`

Make sure your API keys are configured in `api_key.txt` in the project root.

## Step 1: Extract Campaign Parameters

Let's load a real human campaign and extract parameters that we can use to initialize our simulation.

In [2]:
# Set parameters
campaign_name = '20049-age-of-death'#'7921-storm-kings-thunder' #Options: '7426-serpent-isle-private-campaign'#'20049-age-of-death' #'3135-the-head-of-the-serpent' # '89221-banhaven-academy-year-1' # '10391-guardians-of-gridori' '90630-firecat5s-dragon-of-icespire-peak' '1262-firedeath-must-die-closed-play-by-post-open-ogg'
campaign_file = os.path.join(os.getcwd(),"..", "data","raw-human-games", "individual_campaigns", campaign_name + '.json')
include_player_personalities = False
scratchpad = True

# Summarization settings (for long games)
# - summary_chunk_size: turns per summary (set to 0 to disable summarization)
# - verbatim_window: recent turns to keep verbatim (set to None for all turns)
summary_chunk_size = 50  # turns per summary
verbatim_window = 50     # minimum recent turns to keep verbatim

# You can specify different models for different providers:
# model = "claude-3-5-sonnet-20240620"  # Anthropic Claude Sonnet (default)
# model = "claude-3-haiku-20240307"     # Anthropic Claude haiku
# model = "gpt-4o", "gpt-4o-mini"       # OpenAI GPT-4
# model = "gemini-pro"                  # Google Gemini
dnd_dynamics.config.SIMULATION_MODEL = "gemini/gemini-2.0-flash" #"gpt-4o" ## "claude-sonnet-4-5-20250929"  # ##
dnd_dynamics.config.DEFAULT_TEMPERATURE = 1.0  # geimini and gpt: 1.5, claude = 1
print(dnd_dynamics.config.SIMULATION_MODEL)

gemini/gemini-2.0-flash


In [3]:
# Extract parameters from the human campaign
print("📊 Extracting campaign parameters...")
campaign_params = sim.extract_campaign_parameters(campaign_file)

print(f"\n🎯 Campaign Parameters:")
print(f"  Campaign Name: {campaign_params['campaign_name']}")
print(f"  Total Messages: {campaign_params['total_messages']:,}")
print(f"  Number of Players: {campaign_params['num_players']}")
print(f"  Character Names: {campaign_params['character_names']}")
print(f"  Player Names: {campaign_params['player_names']}")
print(f"  Character Classes: {campaign_params['character_classes']}")
print(f"  Character Genders: {campaign_params['character_genders']}")
print(f"  Character Races: {campaign_params['character_races']}")
print(f"  Initial Scenario: {campaign_params['initial_scenario']}")

print('Character Personalities + Sheets')
for i in range(len(campaign_params['character_names'])):
    print(textwrap.fill(f"{campaign_params['character_names'][i]}: {campaign_params['character_personalities'][i]}", width=170))
    print('\n')
    print('Sheet: ')
    if isinstance(campaign_params['character_sheets'][i], dict):
        for key, value in campaign_params['character_sheets'][i].items():
            print(f"{key}: {value}")
    else:
        print(campaign_params['character_sheets'][i])
    print('\n')

print('Player Personalities')
for i in range(len(campaign_params['player_names'])):
    print(
        textwrap.fill(
            f"{campaign_params['player_names'][i]}: {campaign_params['player_personalities'][i]}",
            width=170))
    print('\n')

📊 Extracting campaign parameters...
['Dungeon Master' 'Finlan' None 'Masque' 'Cororin' 'Pizza']
Campaign excerpts for extraction:
=== TURNS 0-99 (Campaign Start) ===
Turn 1 - Dungeon Master: WHAT:                                                                                                                                                                                                                                                                                           DND.................... this is the lost mine of Phandelver intro but it goes into my own twist. with my own city and continents and stuff. brand new continent. same gods and stuff just new government. A man named Gundren Rockseeker sent out a letter to every to every inn in  every  city in  water rite .  the letter states that he can give you a "LARGE" amount of money in return of your help escorting a carriage of precious materials from Solitaire to Aura. the letter clearly said "DON'T OPEN THE BOX!" and to wait a

/Users/annie/anaconda3/envs/py312/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Finlan: ...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Users/annie/anaconda3/envs/py312/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Here are...er_specific_fields=None), input_type=Messag

Finlan:
Level: 5
Class: Barbarian
Race: Halfling
Background: Unknown
Alignment: Unknown
Strength: 16
Dexterity: 14
Constitution: 15
Intelligence: 10
Wisdom: 12
Charisma: 8
Hit Points: Unknown
Armor Class: Unknown
Proficiency Bonus: +3
Saving Throw Proficiencies: Strength, Constitution
Skill Proficiencies: Unknown
Languages: Common, Halfling
Equipment: Unknown
Spells Known: None
Special Abilities: Rage, Halfling Luck
Notes: Likes dwarven ale.

Masque:
Level: 5
Class: Cleric
Race: Githzerai
Background: Unknown
Alignment: Unknown
Strength: 17
Dexterity: 11
Constitution: 13
Intelligence: 14
Wisdom: 14
Charisma: 12
Hit Points: Unknown
Armor Class: Unknown
Proficiency Bonus: +3
Saving Throw Proficiencies: Wisdom, Charisma
Skill Proficiencies: Unknown
Languages: Common, Gith
Equipment: Book
Spells Known: Unknown
Special Abilities: Grave Domain features
Notes: Was studying in Aura, likes to read.

Cororin:
Level: 5
Class: Barbarian
Race: Goliath
Background: Unknown
Alignment: Chaotic Evil
Stre

/Users/annie/anaconda3/envs/py312/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content="Finlan:\...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


## Step 2: Create Characters

Now let's create our AI characters using the player count from the human campaign.

In [4]:
# Create characters based on the campaign parameters
print("🧙 Creating characters...")

characters = sim.create_characters(campaign_params)

print(f"\n✨ Created {len(characters)} characters:")
for char in characters:
    print(f"  🗡️  {char.name} ({char.dnd_class}): {char.personality}")
    print(f"         {char.character_sheet}")

# Get character names for turn sampling
character_names = [char.name for char in characters]
print(f"\n👥 Character roster: {character_names}")

🧙 Creating characters...

✨ Created 5 characters:
  🗡️  Dungeon Master (None): None
         (None,)
  🗡️  Finlan (barbarian): Finlan, a halfling barbarian, appears to be jovial and somewhat easily amused. He enjoys tavern life, particularly dwarven ale, and seeks adventure through posted quests. His reaction to "Pizza Pepperoni" suggests a lighthearted personality with a penchant for finding humor in unexpected places. However, he also displays a reactive temper when he feels overlooked or dismissed, as shown when Gundren seemingly ignored him. He is curious about rewards and keeps a close eye on potentially problematic individuals. Overall, Finlan seems like a friendly, albeit slightly naive, adventurer who enjoys the camaraderie of a group.
         ({'Level': 5, 'Class': 'Barbarian', 'Race': 'Halfling', 'Background': 'Unknown', 'Alignment': 'Unknown', 'Strength': 16, 'Dexterity': 14, 'Constitution': 15, 'Intelligence': 10, 'Wisdom': 12, 'Charisma': 8, 'Hit Points': 'Unknown', 'Armo

## Step 3: Generate Turn Sequence

Create a realistic turn sequence using uniform sampling across all characters.

In [5]:
# Generate a turn sequence 
total_turns = len(campaign_params['character_turns'])-1  # Adjust for desired session length
from_human_game = True

if from_human_game:
    turn_sequence = np.array(campaign_params['character_turns'])[1:total_turns] # start at 1 to remove 1st DM post
else:
    turn_sequence = sim.sample_turn_sequence(
    character_names=character_names,
    total_turns=total_turns,
    method='uniform')

#print(f"\n📋 Turn sequence: {turn_sequence}")

# Show turn distributions
print('Total turn number: ' + str(total_turns))
turn_counts = Counter(turn_sequence)
print(f"\n📊 Turn distribution:")
for char_name, count in turn_counts.items():
    print(f"  {char_name}: {count} turns ({count/total_turns*100:.1f}%)")

Total turn number: 183

📊 Turn distribution:
  Finlan: 31 turns (16.9%)
  Masque: 20 turns (10.9%)
  Dungeon Master: 60 turns (32.8%)
  Cororin: 35 turns (19.1%)
  Pizza: 36 turns (19.7%)


## Step 4: Initialize Game Session

Create the game session manager that will orchestrate our D&D simulation.

In [6]:
# Create game session
print("Initializing game session...")
game_session = sim.GameSession(characters=characters,
                               campaign_name=campaign_name, 
                               scratchpad=scratchpad,
                               summary_chunk_size=summary_chunk_size,
                               verbatim_window=verbatim_window)

print(f"Game session created with {len(characters)} characters")
print(f"Game log initialized (currently {len(game_session.game_log)} events)")


Initializing game session...
📁 Loaded 1 cached basic metrics results
Pre-caching static content...
Game session created with 5 characters
Game log initialized (currently 0 events)


## Step 6: Run the Simulation

Now let's run the complete D&D simulation and watch our AI characters interact!

In [7]:
# Run the simulation
print("🚀 Starting D&D simulation...")
print("\n" + "=" * 60)

# Execute the scenario 
game_session.run_scenario(initial_scenario=campaign_params['initial_scenario'],
                          turn_sequence=turn_sequence,
                          include_player_personalities=include_player_personalities,
                          print_cache=True)

🚀 Starting D&D simulation...

=== SYSTEM PROMPT ===
You are participating in a Dungeons & Dragons play-by-post forum game simulation.  GAME CONTEXT: - This is a turn-based roleplaying game where players control fantasy
characters - Each player posts actions and dialogue for their character in response to game situations - The Dungeon Master (DM) describes scenarios, environments, and
NPC interactions - Players should respond in character, matching typical play-by-post D&D forum style - Responses should include both actions and dialogue as appropriate
for the situation  CRITICAL TURN RESTRICTIONS:   - You are generating EXACTLY ONE turn for your assigned role (either a player character OR the Dungeon Master) - Do NOT
generate responses for other characters or roles  RESPONSE GUIDELINES: - Stay true to your character's personality, abilities, and background - Consider the current
situation and respond appropriately - Match the posting style and tone of play-by-post D&D forums - Include b

/Users/annie/anaconda3/envs/py312/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 7: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='Reasonin...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


[Masque T3] Total tokens: 2021, Prompt toekns: 1927, Completion tokens: 94
   💾 Cached Content: 0, Candidates: 0

=== Masque REASONING ===
Reasoning: The DM has established the initial quest hook. Finlan is already at the table. I should introduce Masque and his interest in the job. I will have Masque ask a
question to gather more information. A concise response is most appropriate given the turn-based nature of the game.  Final response: Masque approaches the table, book in
hand. "Rockseeker, you say? What dangers might we face on this journey?"

=== Masque FINAL RESPONSE ===
Masque approaches the table, book in hand. "Rockseeker, you say? What dangers might we face on this journey?"
Turn counter: 3
--------------------------------------------------
[Dungeon Master T4] Total tokens: 1775, Prompt toekns: 1604, Completion tokens: 171
   💾 Cached Content: 0, Candidates: 0

=== Dungeon Master REASONING ===
Reasoning: The players have responded to the initial quest hook and are waiting at 

KeyboardInterrupt: 

## Export Results

Save the simulation results for further analysis.

In [8]:
# Save simulation results using the built-in save method
# This creates:
# - game-logs/{base}.json (game log with embedded metadata)
# - summaries/{base}.txt (turn summaries)
# - scratchpads/{base}.txt (reasoning if scratchpad=True)
# - metadata_index.json (updated index for filtering)

game_session.save()

💾 Game log saved to: /Users/annie/Code/Repositories/dnd-dynamics/dnd_dynamics/data/llm-games/game-logs/20049-age-of-death_gemini-2.0-flash_no_players_scratch_20260119_110049.json
📝 Summaries saved to: /Users/annie/Code/Repositories/dnd-dynamics/dnd_dynamics/data/llm-games/summaries/20049-age-of-death_gemini-2.0-flash_no_players_scratch_20260119_110049.txt
🧠 Scratchpads saved to: /Users/annie/Code/Repositories/dnd-dynamics/dnd_dynamics/data/llm-games/scratchpads/20049-age-of-death_gemini-2.0-flash_no_players_scratch_20260119_110049.txt
📋 Metadata index updated: /Users/annie/Code/Repositories/dnd-dynamics/dnd_dynamics/data/llm-games/metadata_index.json
